In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
np.random.seed(42)

def get_new_labels(y):
    y_new = LabelEncoder().fit_transform([''.join(str(l)) for l in y])
    return y_new

data_npy = np.load('data/data.npy')
target_npy = np.load('data/target.npy')
y_new = get_new_labels(np.where(target_npy > 0, 1, 0))


In [4]:
splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(data_npy, y_new))

c:\Users\atuan\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [40]:
(train_dx, test_dx) = splits[0]

In [41]:
unique, counts = np.unique(target_npy[train_dx], return_counts=True)
unique, counts

(array([0, 1, 2, 3, 4, 5], dtype=int64),
 array([14320,   305,   333,   356,   820,  1566], dtype=int64))

In [42]:
unique, counts = np.unique(target_npy[test_dx], return_counts=True)
unique, counts

(array([0, 1, 2, 3, 4, 5], dtype=int64),
 array([3580,   84,   70,   92,  208,  394], dtype=int64))